## Quickstart

This notebook shows how to get started with using x.infer.

x.infer relies on PyTorch and torchvision, so make sure you have it installed on your system.

In [ ]:
!pip install -Uqq torch torchvision

Let's check if PyTorch is installed by checking the version.

In [ ]:
import torch

torch.__version__

Also let's check if CUDA is available.

In [ ]:
torch.cuda.is_available()

x.infer relies on various optional dependencies like transformers, ultralytics, timm, etc.
You don't need to install these dependencies if you don't want to. Just install x.infer with the dependencies you want.

For example, if you'd like to use models from the transformers library, you can install the `transformers` extra:


```bash
pip install -Uqq "xinfer[transformers]"
```

To install all the dependencies, you can run:
```bash
pip install -Uqq "xinfer[all]"
```



In [ ]:
!pip install -Uqq "xinfer[transformers]"

It's recommended to restart the kernel once all the dependencies are installed.

In [ ]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(restart=True)

Once completed, let's import x.infer, check the version and list all the models available.

In [1]:
import xinfer

print(xinfer.__version__)
xinfer.list_models()

/home/dnth/mambaforge-pypy3/envs/xinfer-test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.0.5


                            Available Models                             
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Implementation ┃ Model ID                       ┃ Input --> Output    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ transformers   │ Salesforce/blip2-opt-6.7b-coco │ image-text --> text │
│ transformers   │ Salesforce/blip2-flan-t5-xxl   │ image-text --> text │
│ transformers   │ Salesforce/blip2-opt-6.7b      │ image-text --> text │
│ transformers   │ Salesforce/blip2-opt-2.7b      │ image-text --> text │
│ transformers   │ vikhyatk/moondream2            │ image-text --> text │
└────────────────┴────────────────────────────────┴─────────────────────┘

Because we only installed the `transformers` extra, we can only use models from the `transformers` library.

You can pick any model from the list of models available.
Let's create a model from the `vikhyatk/moondream2` model. We can optionally specify the device and dtype. By default, the model is created on the CPU and the dtype is `float32`.

In [2]:
model = xinfer.create_model("vikhyatk/moondream2", device="cuda", dtype="float16")

2024-10-17 01:47:53.149 | INFO     | xinfer.transformers.moondream:__init__:24 - Model: vikhyatk/moondream2
2024-10-17 01:47:53.150 | INFO     | xinfer.transformers.moondream:__init__:25 - Revision: 2024-08-26
2024-10-17 01:47:53.150 | INFO     | xinfer.transformers.moondream:__init__:26 - Device: cuda
2024-10-17 01:47:53.150 | INFO     | xinfer.transformers.moondream:__init__:27 - Dtype: float16
PhiForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify y

Now that we have the model, let's infer an image.

In [ ]:
from PIL import Image
import requests

image_url = "https://raw.githubusercontent.com/vikhyat/moondream/main/assets/demo-1.jpg"
Image.open(requests.get(image_url, stream=True).raw)


You can pass in a url or the path to an image file.

In [22]:
image = "https://raw.githubusercontent.com/vikhyat/moondream/main/assets/demo-1.jpg"
prompt = "Caption this image."

model.infer(image, prompt)

'An anime-style illustration depicts a young girl with white hair and green eyes, wearing a white jacket and holding a large burger, seated at a table with a plate of food in front of her.'

If you'd like to generate a longer caption, you can do so by setting the `max_new_tokens` parameter. You can also pass in any generation parameters supported by the `transformers` library.

In [25]:
image = "https://raw.githubusercontent.com/vikhyat/moondream/main/assets/demo-1.jpg"
prompt = "Caption this image highlighting the focus of the image and the background in detail."

model.infer(image, prompt, max_new_tokens=500)

'The image depicts a young girl with long, white hair and blue eyes sitting at a table, holding a large burger in her hands. The background shows a dimly lit room with a window, suggesting an indoor setting.'

If you'd like to see the inference stats, you can do so by calling the `print_stats` method. This might be useful if you're running some sort of benchmark on the inference time.

In [31]:
model.stats.print_stats()

                    Model Stats                    
╭───────────────────────────┬─────────────────────╮
│ Attribute                 │ Value               │
├───────────────────────────┼─────────────────────┤
│ Model ID                  │ vikhyatk/moondream2 │
│ Device                    │ cuda                │
│ Dtype                     │ torch.float16       │
│ Number of Inferences      │ 15                  │
│ Total Inference Time (ms) │ 9590.0959           │
│ Average Latency (ms)      │ 639.3397            │
╰───────────────────────────┴─────────────────────╯

Finally, you can also run batch inference. You'll have to pass in a list of images and prompts.

In [32]:
model.infer_batch([image, image], [prompt, prompt])

['The image depicts a young girl with long, white hair and blue eyes sitting at a table, holding a large burger in her hands. The background shows a cozy indoor setting with a window and a chair visible.',
 'The image depicts a young girl with long, white hair and blue eyes sitting at a table, holding a large burger in her hands. The background shows a cozy indoor setting with a window and a chair visible.']

: 

That's it! You've successfully run inference with x.infer. 

Hope this simplifies the process of running inference with your favorite computer vision models!

<div align="center">
    <img src="https://raw.githubusercontent.com/dnth/x.infer/refs/heads/main/assets/github_banner.png" alt="x.infer" width="600"/>
    <br />
    <br />
    <a href="https://dnth.github.io/x.infer" target="_blank" rel="noopener noreferrer"><strong>Explore the docs »</strong></a>
    <br />
    <a href="#quickstart" target="_blank" rel="noopener noreferrer">Quickstart</a>
    ·
    <a href="https://github.com/dnth/x.infer/issues/new?assignees=&labels=Feature+Request&projects=&template=feature_request.md" target="_blank" rel="noopener noreferrer">Feature Request</a>
    ·
    <a href="https://github.com/dnth/x.infer/issues/new?assignees=&labels=bug&projects=&template=bug_report.md" target="_blank" rel="noopener noreferrer">Report Bug</a>
    ·
    <a href="https://github.com/dnth/x.infer/discussions" target="_blank" rel="noopener noreferrer">Discussions</a>
    ·
    <a href="https://dicksonneoh.com/" target="_blank" rel="noopener noreferrer">About</a>
</div>